In [92]:
import pandas as pd

In [93]:
data = pd.read_csv("Datasets and Python codes-20221223/mul_reg.csv")
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [94]:
state = pd.get_dummies(data['State'])
data = data.join(state)
data = data.drop('State', axis=1)

In [95]:
data.head()

,R&D Spend,Administration,Marketing Spend,Profit,California,Florida,New York
0,165349.20,136897.80,471784.10,192261.83,0,0,1
1,162597.70,151377.59,443898.53,191792.06,1,0,0
2,153441.51,101145.55,407934.54,191050.39,0,1,0
3,144372.41,118671.85,383199.62,182901.99,0,0,1
4,142107.34,91391.77,366168.42,166187.94,0,1,0


Find out VIF Values:

Find maximum VIF values and remove from the dataset.

<u> Note:</u>
Decision Tree Regn, Support Vector Regn -> No need to check VIF

Create an intercept = '1' column, since y = b0(1) + b1*x1 + .... + bn*xn

In [96]:
x = data.iloc[:, [0, 1, 2, 4, 5, 6]]

In [102]:
vif = pd.DataFrame()
vif['variable'] = x.columns
vif

,variable
0,R&D Spend
1,Administration
2,Florida
3,New York


In [103]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Use '\' (backward slash) in statement
# to continue command in next line

vif['variance'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]

# v_i_f() -> Will take first 'constant' column as 'intercept'. 
# In data, If no intercept constant given, it will take 1st indep. var. as intercept & ignore.

# shape = (<numOfRows>, <numOfCols>)

In [104]:
vif

,variable,variance
0,R&D Spend,3.917998
1,Administration,4.863405
2,Florida,1.881915
3,New York,1.909248


'inf' = highly correllated. 
Do not remove all 3 columns. Remove only 1 column first.

In [100]:
x = x.drop('Marketing Spend', axis=1)

In [101]:
x = x.drop('California', axis=1)

In [105]:
y = data.iloc[:, 3].values

In [106]:
# x = x.values
# Will give values in DataFrame

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
# Note: train_test_split() will internally shuffle data

In [119]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# Since 'x_test' is a new occurrence, use mean and std deviation of training data.
# Since 'fit' was done in 'x_train', 'x_test' will use the same parameters.
# Reason for 1st Split, and 2nd Scaling -> To separate 'train' data mean and std dev from 'test'

x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
y_train = sc.fit_transform(y_train.reshape(-1, 1))
y_test = sc.transform(y_test.reshape(-1, 1))

In [120]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

In [121]:
print(reg.intercept_)
print(reg.coef_)

# If you don't remove 'intercept', it will be considered as indep variable.
# Model understood that 'intercept' had no value to output, so gave value = '0'

[-3.4824468e-17]
[[ 0.97015962  0.01138085  0.0117517  -0.00668633]]


In [122]:
from sklearn.metrics import r2_score, mean_squared_error

print("R2 Score: ", r2_score(y_test, y_pred))

import math

print("MSE: ", math.sqrt(mean_squared_error(y_test, y_pred)))

R2 Score:  0.947720767317498
MSE:  0.22864652344285064
